In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df0=pd.read_csv('/content/Bengaluru_House_Data.csv')
df0.drop(['availability','society'],axis=1,inplace=True)
df=df0.copy()
df.fillna(df.mean(),inplace=True)
df.dropna(inplace=True)
df.groupby('size')['size'].count()
df1=df.copy()
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None
df['Bedrooms']=df['size'].str.split(' ').apply(lambda x: x[0])
df['hk']=df['size'].str.split(' ').apply(lambda x: x[1])
df.drop(['size'],axis=1,inplace=True)
df.reset_index(inplace=True)
df['HK'] = df['hk'] == 'BHK'
df["HK"] = df["HK"].astype(int)
df.drop(['hk'],axis=1,inplace=True)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df.area_type = le.fit_transform(df.area_type)
df.location = le.fit_transform(df.location)
df.drop(['index'],axis=1,inplace=True)
df['total_sqft']=df['total_sqft'].apply(convert_sqft_to_num)
df = df[df.total_sqft.notnull()]
price=pd.DataFrame(df.price)
df.drop(['price'],axis=1,inplace=True)
from sklearn.model_selection import train_test_split
price=np.ravel(price)
x_train,x_test,y_train,y_test=train_test_split(df,price,test_size=0.2,random_state=0)
from sklearn.tree import DecisionTreeRegressor
m=0
r=range(20)
for k in r:
  dtregressor = DecisionTreeRegressor(random_state=k)
  dtregressor.fit(df,price)
  if m<dtregressor.score(x_test,y_test):
    m=dtregressor.score(x_test,y_test)
print(m)

<ipython-input-2-6db7f1bf4f83>:8: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(),inplace=True)
<ipython-input-2-6db7f1bf4f83>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['price'],axis=1,inplace=True)


0.9970215446060327


In [3]:
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(dtregressor,f)

In [5]:
import json
columns = {
    'data_columns' : [col.lower() for col in df.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

In [ ]:
#wb stands for binary data
#f is file
# and 'banglore_home_prices_model.pickle' will be the name of the file 